In [1]:
%load_ext autoreload
%autoreload 2
import sys, os, torch, datetime, time
import numpy as np
sys.path.append("..")
import rfi_ml
import matplotlib.pyplot as plt

In [2]:
Ntrain = 100000 #the size of the training set
Ntest = 10
sigma_low = 20
sigma_high = 50
Nepochs = 30
Nrfi = 1
Np = 2**10 #1k
hidden_dim = 2**10
#Np = 2**10 #1k
#hidden_dim = 1024
hidden_dim_2 = 512
z_dim = 16
Nfft = Np // 2 + 1
#Pk = np.ones(Nfft)
Pk = None
#lamb = 0.9 #lambda, scaling down Gaussianized amplitude
np.random.seed(0) #for debugging
torch.manual_seed(0) #more debugging
start_time = time.time()

t = rfi_ml.ToyGenerator(Np=Np, Pk=Pk)
g = torch.stack([torch.from_numpy(t.getGaussian()) for i in range(Ntrain+Ntest)])
#Pk2 = None
#t2 = rfi_ml.ToyGenerator(Np=Np, Pk=Pk2)
#g2 = torch.stack([torch.from_numpy(t2.getGaussian()) for i in range(Ntest)]) 
rfi_array = np.zeros((Ntrain+Ntest, Np), dtype=float)
rfi_pwr = np.zeros(Ntrain+Ntest, dtype=float)
for i in range(Ntrain + Ntest):
#for i in range(Ntrain):
    rfi_timestream = np.zeros(Np)
    rfi_timestream_pwr = 0
    for j in range(Nrfi):
        rfi_timestream_tmp, rfi_timestream_pwr_tmp = t.getNonGaussianLocalized(sigma=(sigma_low, sigma_high)) 
        #rfi_timestream_tmp, rfi_timestream_pwr_tmp = t.getNonGaussianNonlocalized(freq = ((2*np.pi)/(Np), (4*np.pi)/Np), Pflip=1.0) 
        rfi_timestream += rfi_timestream_tmp
        rfi_timestream_pwr += rfi_timestream_pwr_tmp
    rfi_array[i,:] += rfi_timestream
    rfi_pwr[i] += rfi_timestream_pwr
ng = torch.from_numpy(rfi_array)
#ng = torch.stack([torch.from_numpy(t.getNonGaussianLocalized(sigma=(sigma_low, sigma_high))) for i in range(Ntrain+Ntest)])


In [3]:
n = rfi_ml.RFIDetect(Np, Nepochs = Nepochs, z_dim = z_dim, hidden_dim = hidden_dim, hidden_dim_2 = hidden_dim_2)

In [4]:
n.train(g[:Ntrain], ng[:Ntrain])

[  0/30][  0/3125]	Loss: 0.0011554866
[  0/30][100/3125]	Loss: 0.0011087987
[  0/30][200/3125]	Loss: 0.0010771173
[  0/30][300/3125]	Loss: 0.0009952807
[  0/30][400/3125]	Loss: 0.0010544175
[  0/30][500/3125]	Loss: 0.0010818513
[  0/30][600/3125]	Loss: 0.0010803404
[  0/30][700/3125]	Loss: 0.0011694182
[  0/30][800/3125]	Loss: 0.0010499218
[  0/30][900/3125]	Loss: 0.0012199137
[  0/30][1000/3125]	Loss: 0.0011357367
[  0/30][1100/3125]	Loss: 0.0010901365
[  0/30][1200/3125]	Loss: 0.0010576543
[  0/30][1300/3125]	Loss: 0.0010530038
[  0/30][1400/3125]	Loss: 0.0010186443
[  0/30][1500/3125]	Loss: 0.0010663628
[  0/30][1600/3125]	Loss: 0.0011028100
[  0/30][1700/3125]	Loss: 0.0010209370
[  0/30][1800/3125]	Loss: 0.0011503323
[  0/30][1900/3125]	Loss: 0.0010417337
[  0/30][2000/3125]	Loss: 0.0010903180
[  0/30][2100/3125]	Loss: 0.0011171678
[  0/30][2200/3125]	Loss: 0.0010694938
[  0/30][2300/3125]	Loss: 0.0010898588
[  0/30][2400/3125]	Loss: 0.0009970930
[  0/30][2500/3125]	Loss: 0.0011128

[  6/30][2450/3125]	Loss: 0.0003867261
[  6/30][2550/3125]	Loss: 0.0003800053
[  6/30][2650/3125]	Loss: 0.0003837532
[  6/30][2750/3125]	Loss: 0.0003782465
[  6/30][2850/3125]	Loss: 0.0003728264
[  6/30][2950/3125]	Loss: 0.0003772415
[  6/30][3050/3125]	Loss: 0.0003836720
[  7/30][ 25/3125]	Loss: 0.0003808896
[  7/30][125/3125]	Loss: 0.0003788798
[  7/30][225/3125]	Loss: 0.0003761784
[  7/30][325/3125]	Loss: 0.0003714967
[  7/30][425/3125]	Loss: 0.0003747154
[  7/30][525/3125]	Loss: 0.0003813180
[  7/30][625/3125]	Loss: 0.0003734805
[  7/30][725/3125]	Loss: 0.0003785226
[  7/30][825/3125]	Loss: 0.0003757614
[  7/30][925/3125]	Loss: 0.0003705744
[  7/30][1025/3125]	Loss: 0.0003783538
[  7/30][1125/3125]	Loss: 0.0003767922
[  7/30][1225/3125]	Loss: 0.0003746449
[  7/30][1325/3125]	Loss: 0.0003785660
[  7/30][1425/3125]	Loss: 0.0003744684
[  7/30][1525/3125]	Loss: 0.0003767457
[  7/30][1625/3125]	Loss: 0.0003772340
[  7/30][1725/3125]	Loss: 0.0003749193
[  7/30][1825/3125]	Loss: 0.0003715

[ 13/30][1775/3125]	Loss: 0.0003670077
[ 13/30][1875/3125]	Loss: 0.0003678610
[ 13/30][1975/3125]	Loss: 0.0003660051
[ 13/30][2075/3125]	Loss: 0.0003712475
[ 13/30][2175/3125]	Loss: 0.0003628123
[ 13/30][2275/3125]	Loss: 0.0003686561
[ 13/30][2375/3125]	Loss: 0.0003698206
[ 13/30][2475/3125]	Loss: 0.0003667510
[ 13/30][2575/3125]	Loss: 0.0003704034
[ 13/30][2675/3125]	Loss: 0.0003654056
[ 13/30][2775/3125]	Loss: 0.0003723285
[ 13/30][2875/3125]	Loss: 0.0003691446
[ 13/30][2975/3125]	Loss: 0.0003646264
[ 13/30][3075/3125]	Loss: 0.0003617811
[ 14/30][ 50/3125]	Loss: 0.0003657677
[ 14/30][150/3125]	Loss: 0.0003640788
[ 14/30][250/3125]	Loss: 0.0003621126
[ 14/30][350/3125]	Loss: 0.0003640322
[ 14/30][450/3125]	Loss: 0.0003708336
[ 14/30][550/3125]	Loss: 0.0003626038
[ 14/30][650/3125]	Loss: 0.0003651374
[ 14/30][750/3125]	Loss: 0.0003614429
[ 14/30][850/3125]	Loss: 0.0003712215
[ 14/30][950/3125]	Loss: 0.0003618033
[ 14/30][1050/3125]	Loss: 0.0003719608
[ 14/30][1150/3125]	Loss: 0.0003652

[ 20/30][1100/3125]	Loss: 0.0003588521
[ 20/30][1200/3125]	Loss: 0.0003618457
[ 20/30][1300/3125]	Loss: 0.0003584780
[ 20/30][1400/3125]	Loss: 0.0003606494
[ 20/30][1500/3125]	Loss: 0.0003629113
[ 20/30][1600/3125]	Loss: 0.0003611904
[ 20/30][1700/3125]	Loss: 0.0003582619
[ 20/30][1800/3125]	Loss: 0.0003634095
[ 20/30][1900/3125]	Loss: 0.0003657043
[ 20/30][2000/3125]	Loss: 0.0003567387
[ 20/30][2100/3125]	Loss: 0.0003587359
[ 20/30][2200/3125]	Loss: 0.0003571830
[ 20/30][2300/3125]	Loss: 0.0003594505
[ 20/30][2400/3125]	Loss: 0.0003714572
[ 20/30][2500/3125]	Loss: 0.0003649749
[ 20/30][2600/3125]	Loss: 0.0003642328
[ 20/30][2700/3125]	Loss: 0.0003589952
[ 20/30][2800/3125]	Loss: 0.0003612474
[ 20/30][2900/3125]	Loss: 0.0003626850
[ 20/30][3000/3125]	Loss: 0.0003656832
[ 20/30][3100/3125]	Loss: 0.0003679312
[ 21/30][ 75/3125]	Loss: 0.0003601546
[ 21/30][175/3125]	Loss: 0.0003544152
[ 21/30][275/3125]	Loss: 0.0003557564
[ 21/30][375/3125]	Loss: 0.0003616040
[ 21/30][475/3125]	Loss: 0.00

[ 27/30][425/3125]	Loss: 0.0003578943
[ 27/30][525/3125]	Loss: 0.0003570293
[ 27/30][625/3125]	Loss: 0.0003616335
[ 27/30][725/3125]	Loss: 0.0003594394
[ 27/30][825/3125]	Loss: 0.0003570741
[ 27/30][925/3125]	Loss: 0.0003584017
[ 27/30][1025/3125]	Loss: 0.0003658102
[ 27/30][1125/3125]	Loss: 0.0003638566
[ 27/30][1225/3125]	Loss: 0.0003595846
[ 27/30][1325/3125]	Loss: 0.0003610208
[ 27/30][1425/3125]	Loss: 0.0003588049
[ 27/30][1525/3125]	Loss: 0.0003617179
[ 27/30][1625/3125]	Loss: 0.0003546186
[ 27/30][1725/3125]	Loss: 0.0003565360
[ 27/30][1825/3125]	Loss: 0.0003626433
[ 27/30][1925/3125]	Loss: 0.0003552790
[ 27/30][2025/3125]	Loss: 0.0003636165
[ 27/30][2125/3125]	Loss: 0.0003592653
[ 27/30][2225/3125]	Loss: 0.0003618014
[ 27/30][2325/3125]	Loss: 0.0003616289
[ 27/30][2425/3125]	Loss: 0.0003582725
[ 27/30][2525/3125]	Loss: 0.0003590183
[ 27/30][2625/3125]	Loss: 0.0003591161
[ 27/30][2725/3125]	Loss: 0.0003656511
[ 27/30][2825/3125]	Loss: 0.0003584889
[ 27/30][2925/3125]	Loss: 0.000

In [5]:
start_test = time.time()
testout, activation = n.evaluate(g[Ntrain:], ng[Ntrain:], rfi_pwr[Ntrain:])
#testout = n.evaluate(g2, ng[Ntrain:])
end_test = time.time()

Epochs:  30
N tests:  10
Length of timestream:  1024
Timestream Obs-Expect RMS:  [0.00386054 0.00396236 0.00292359 0.00353652 0.00568452 0.00407266
 0.00335284 0.00374614 0.00283846 0.00348104]
Avg RMS for all tests:  0.003745867935639042 

Fraction of RFI power remaining:  [0.01574743 0.02691771 0.02003146 0.04562022 0.03208998 0.01345117
 0.01964941 0.01079843 0.02012309 0.01660337]
Remainder as fraction of signal power:  [0.0398466  0.04267289 0.02455247 0.03441439 0.08714421 0.04711308
 0.02943156 0.04042605 0.02368367 0.03563649]
Avg of RFI power remaining:  0.02210322684667846
Input RFI Timestream RMS:  [0.030764   0.02415101 0.02065667 0.01655761 0.03173288 0.03511542
 0.02391871 0.0360499  0.02000949 0.02701536]
Avg RFI RMS:  0.026597105129356043
Input Gaussian Timestream RMS:  [0.0193398  0.01918132 0.01865817 0.01906366 0.01925639 0.01876322
 0.01954366 0.01863175 0.01844416 0.01844002]
Avg Gaussian RMS:  0.018932213798650045
Signal to Noise Ratio:  1.9736309149792408
Signal 

In [6]:
n.plot_eval(testout, g[Ntrain:], ng[Ntrain:], activation)

Saving file...rfi_ml/2022-09-29_13-12-26_epoch_0000030_test_0.pdf
Saving file...rfi_ml/2022-09-29_13-12-26_overplot_test_0.pdf
Saving file...rfi_ml/2022-09-29_13-12-26_residual_rfi_test_0.pdf
Saving file...rfi_ml/2022-09-29_13-12-26_RFI_subtracted_test_0.pdf
Saving file...rfi_ml/2022-09-29_13-12-26_diff_test_0.png
Saving file...rfi_ml/2022-09-29_13-12-26_frac_err_test_0.png
Median Frac Diff:  0.004456579723207825
Saving file...rfi_ml/2022-09-29_13-12-26_epoch_0000030_test_1.pdf
Saving file...rfi_ml/2022-09-29_13-12-26_overplot_test_1.pdf
Saving file...rfi_ml/2022-09-29_13-12-26_residual_rfi_test_1.pdf
Saving file...rfi_ml/2022-09-29_13-12-26_RFI_subtracted_test_1.pdf
Saving file...rfi_ml/2022-09-29_13-12-26_diff_test_1.png
Saving file...rfi_ml/2022-09-29_13-12-26_frac_err_test_1.png
Median Frac Diff:  0.0020926313514737387
Saving file...rfi_ml/2022-09-29_13-12-26_epoch_0000030_test_2.pdf
Saving file...rfi_ml/2022-09-29_13-12-26_overplot_test_2.pdf
Saving file...rfi_ml/2022-09-29_13-12-

In [7]:
print("Runtime: ", time.time()-start_time)
print("Evaltime: ", end_test-start_test)

Runtime:  430.3960802555084
Evaltime:  0.002950429916381836
